
## Gaggle Competition for Knowledge - Predict Future Sales

https://www.kaggle.com/c/competitive-data-science-predict-future-sales

Marcos Vanine Nader

vanine6@hotamil.com

August/2019


### Introduction

This a Notebook with the solution for the Predict Future Sales (Kaggle competition for knowledge).

The topics sequence is:

1. Imports, global settings and basic funtions. 
2. Read data and perform minor adjusts
3. Feaure engineering: for each feature the analysis is performed and following the conversion into the data structure to be used to fit the models.
4. Build the train, validation and test datasets for first level models
5. Instantiate, fit and calculate the predictions for the first level models
6. Build the dataset for ensemble models
7. Split into train, valid and test datasets and instantiate, fit and calculate the predictions for ensemble models
8. Generate the submission file. 

Some of public kernels were used for studies. The most influent in this development is: https://www.kaggle.com/dlarionov/feature-engineering-xgboost - thanks to Denis Larionov

### About the models

The final model is a ensemble of two other models: (M1) xgboost and (M2) lightGBM

These two models are trained for three sets of pairs (train, valid) of datasets. These datasets are derived from tha original train dataset with the following characterirstics:

(C0) train dataset with the records before month 29 and validation equal 29

(C1) train dataset with the records before month 30 and validation equal 30

(C2) train dataset with the records before month 31 and validation equal 31

(C3) train dataset with the records before month 32 and validation equal 32

The ensemble will have as train dataset the predictions on validation of C1, C2 and C3 for both models. 
So the features for the ensemble model are M1_C0, M1_C1, M1_C2, M1_C3, M2_C0, M2_C1, M2_C2, M3_C3 and target the input dataset target for month 33.

The ensemble model chosen is linear regression applied in this new dataset.

### The process of generating the models and the submission file

This script is prepared to run one model at each time - this restriction is because the memory necessary to run the models.

Step 1: set the model_option="xgb", model_name="M1C0" and validation_month=29 - execute Run All - the model M1C0 will be saved in the directory "models_to_ensemble"

Step 2: same as above with model_name="M1C1" and validation_month=30

Step 3: same as above with model_name="M1C2" and validation_month=31

Step 4: same as above with model_name="M1C3" and validation_month=32

Step 5: same as above with model_option="lgb", model_name="M2C0" and validation_month=29

Step 6: same as above with model_name="M2C1" and validation_month=30

Step 7: same as above with model_name="M2C2" and validation_month=31

Step 8: same as above with model_name="M2C3" and validation_month=32

Step 9: set the model_option="ensemble", ens_first_level_models = list of the 6 model names above and ens_validation_month = 33

The submission file will be stored in the directory "output" with a name: "./output/submission_" + model_option + '_' + sct2() + ".csv" where sct2() returns a string with the current timedate.

### 1. Import, global settings and basic functiona

In [1]:
# 1.1 General import declarations - specific imports are in the associated cells
# ------------------------------------------------------------------------------
import pandas as pd 
import numpy as np 
#
from itertools import product # cartesian product
#
import os 
import time 
from IPython.display import display 
#
%matplotlib inline
import matplotlib.pyplot as plt
#
import datetime
#
import warnings
#
from scipy.sparse import csr_matrix
#
from sklearn.preprocessing import OneHotEncoder
#
import psutil, gc

In [2]:
import sklearn

print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.21.3.


In [3]:
# 1.2. Settings - part 1
# ----------------------
print (time.strftime("%d/%m/%y %H:%M:%S"))
# General settings 

# Feature Engineering Settings
# lags
complex_ll = [1, 2, 3, 6, 12]
simple_ll = [1]
medium_ll = [1, 2]
item_cnt_month_ll = complex_ll
date_avg_item_cnt_ll = simple_ll 
date_item_avg_item_cnt_ll = complex_ll
date_shop_avg_item_cnt_ll = complex_ll
date_cat_avg_item_cnt_ll = medium_ll
date_shop_cat_avg_item_cnt_ll = simple_ll
date_shop_item_avg_price_ll = medium_ll
first_time_cat_ll = complex_ll
first_time_shop_ll = complex_ll
# -- max number of lags - used  to delete the first rows
max_lag = 12
print("max_lag = ", max_lag)
# Switches - to decide inlcude or not a new feature
#
# Note that when setting a "include" variable in the list below, it is necessary to review the corresponding lag variable 
# in the list above
#
include_city_code = True                     # Include city code as a feature to be trained
include_item_cnt_lag = True                  # Item count per month lag
include_date_avg_item_cnt = True             # Include average item count per month 
include_date_item_avg_item_cnt = True        # (ok) Include average item count per item per month
include_date_shop_avg_item_cnt = True        # (ok) Include average item count per shop per month
include_date_cat_avg_item_cnt = True         # (ok 1) Include average item count per category per month
include_date_shop_cat_avg_item_cnt = True    # (ok 1) Include average item count per category per shop per month
transform_price_log10 = True                 # Transform price to log10(price)
include_date_price_avg_item = True           # Include average price per item per month
month_onehotencoder = True                   # Month of year as one_hot_encoded (otherwise it will be added as a integer 1-12)
include_item_shop_first_sale = True          # Include time in months since first sale: for each pair shop/item and for each item
include_date_cat_avg_ft = True               # (ok) Include average item count per category for the first time 
include_date_shop_avg_ft = False             # Include average item count per shop for the first time
# td-idf for the item name - the variable below is the size of the vector where the item name will be transformed
include_item_name = True                     # Include item name transformed by td-idf rules
item_name_vec_size = 60                      # should be 60? This is what it was used in the best result run
# Drop some features- 
drop_date_block_num = False                  # Drop data_block_num
# We have included mean encode - is it necessary to keep the original categoty variables?
drop_item_id = False                         # Drop item_id
drop_shop_id = False                         # Drop shop_id

print (time.strftime("%d/%m/%y %H:%M:%S"))



24/08/19 21:38:12
max_lag =  12
24/08/19 21:38:12


In [4]:
# 1.3 Settings - part 2 - Operation mode and model option (algorithm type)   
# ------------------------------------------------------------------------
first_level_model_options = ["xgb", "lgb"] 
ensemble_model_options = ["ens_sgd", "ens_mean"]
# 
model_option = "ens_mean"          # "xgb" xgboost - "lgb" lightm GBM
                                  # "ens_sgd" - linear regression and "ens_mean" - arithmetic mean
# first level
model_name = "M2C0"               # name of the first level model to be fitted and saved
validation_month = 29             # validation model for first level models

# ensemble
                                  # list of models to be used in the ensemble
ens_first_level_models = ["M1C0", "M1C1", "M1C2", "M1C3", "M2C0", "M2C1", "M2C2", "M2C3"]                                 
ens_validation_month = 33         # validation_month for the ensemble

# Deprecated
only_train_test = False           # run the regressor only with train data (no validation) and submit the result - so that, we
                                  # train for the 33 months and predict the next (34th)
n_estimators_only_train_test = 39 # number of estimators when training without validation 


In [5]:
# 1.4 Basic functions 
# -------------------
#-- print the current time (pct)
def pct():
    print (time.strftime("%d/%m/%y %H:%M:%S"))
#-- return a string with the current time    
def sct():
    return (time.strftime("%d/%m/%y %H:%M:%S"))
def sct2():
    return (time.strftime("%y%m%d_%H%M%S"))
#-- Class TimeServ - controls the time duration of a service
import time
class TimeServ (object):
    def __init__(self, name=""):
        self.name = name
        self.start_time = time.time()
    def exit(self):
        duration = time.time() - self.start_time
        if duration > 60.0:
            print (self.name, "execution time {:.3f} min.".format(duration/60))
        else:
            print (self.name, "execution time {:.3f} sec.".format(duration))
        return duration
            
# -- Test
s = TimeServ ("loop_10_milion")
for i in range(10000000):
    j=1
s.exit()
#-- print the string given in "expr" parameter and its evaluation  
def dprint(*expr, sep=""):
    for e in expr:
        print(e, "=", sep, eval(e))
#-- define end of line in order to facilitate the use of dprint
EOL = "\n"        
#-- test
pct()
dprint("sct()")
dprint("sct2()")
a = 6
dprint("a")

loop_10_milion execution time 2.249 sec.
24/08/19 21:38:14
sct() =  24/08/19 21:38:14
sct2() =  190824_213814
a =  6


In [6]:
pct()
print("Virtual Memory")
print(psutil.virtual_memory())

24/08/19 21:38:14
Virtual Memory
svmem(total=12748468224, available=7220510720, percent=43.4, used=5527957504, free=7220510720)


#### 2. Read data and perform minor adjusts

In [7]:
# 2.1. Read data function
# -----------------------
# read_file function
pct()
def read_file (file_name, data_dir = "./input"):
    # -- Read the file identified by "file_name" in the directoty "data_dir", 
    # -- return a pndas dataframe
    data = pd.read_csv (os.path.join(data_dir, file_name))  
    print(file_name)
    print("shape of", file_name, data.shape)
    print(data.dtypes)
    display(data.head())
    return data

print("read_file compiled")


24/08/19 21:38:14
read_file compiled


In [8]:
# 2.2. Read Files
pct()
# Read all files 

train_ds = read_file("sales_train_v2.csv")
test_ds = read_file("test.csv")
sample_submission_ds = read_file("sample_submission.csv")
items_ds = read_file("items.csv")
item_categories_ds = read_file ("item_categories.csv")
shops_ds = read_file("shops.csv")

24/08/19 21:38:14
sales_train_v2.csv
shape of sales_train_v2.csv (2935849, 6)
date               object
date_block_num      int64
shop_id             int64
item_id             int64
item_price        float64
item_cnt_day      float64
dtype: object


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


test.csv
shape of test.csv (214200, 3)
ID         int64
shop_id    int64
item_id    int64
dtype: object


,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


sample_submission.csv
shape of sample_submission.csv (214200, 2)
ID                  int64
item_cnt_month    float64
dtype: object


...

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


item_categories.csv
shape of item_categories.csv (84, 2)
item_category_name    object
item_category_id       int64
dtype: object


,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


shops.csv
shape of shops.csv (60, 2)
shop_name    object
shop_id       int64
dtype: object


,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
# 2.3 Delete outliers
# -------------------
pct()
print(train_ds.shape)
train_ds = train_ds[train_ds.item_price<100000]    # delete price >=1000000
print(train_ds.shape)
train_ds = train_ds[train_ds.item_price>0]         # delete price <0
print(train_ds.shape)
train_ds = train_ds[train_ds.item_cnt_day<1001]    # delete item_cnt_day > 1000
print(train_ds.shape)

24/08/19 21:38:18
(2935849, 6)
(2935848, 6)
(2935847, 6)
(2935846, 6)


### 3. Fetaure engineering 


In [10]:
# 3.1 Build the dataset "both_ds" with train and test data
#---------------------------------------------------------
# What identify the test dataset in "both_ds" is the column "date_block_num": 34 is for test.
# 3.1.1 Build a 3-D matrix with indexes: (month, shop, item)
pct()
both_list = []
key_cols = ["date_block_num","shop_id","item_id"]
for i in range(34):
    sales = train_ds[train_ds.date_block_num == i]
    both_list.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype="int16"))
# -- Create the dataframe that will contain both data (train and test)    
both_ds = pd.DataFrame(np.vstack(both_list), columns=key_cols)
# -- Sort the keys
both_ds.sort_values(key_cols, inplace=True)
#
print(type(both_ds))
print(both_ds.shape)
print(both_ds.dtypes)
display(both_ds[10000:10005])
pct()

24/08/19 21:38:19
<class 'pandas.core.frame.DataFrame'>
(10913804, 3)
date_block_num    int16
shop_id           int16
item_id           int16
dtype: object


,date_block_num,shop_id,item_id
152779,0,1,6028
153185,0,1,6030
150410,0,1,6031
146562,0,1,6033
146563,0,1,6036


24/08/19 21:38:39


In [11]:
# 3.1.2 Create a column with the item quantity sold  per shop per month 
# (this is the target - not applicable to test dataset)
# ------------------------------------------------------------------
pct()
# Group the train dataset to have the quantities sold by (month, shop, item)
group = train_ds.groupby(key_cols).agg({"item_cnt_day": ["sum"]})
group.columns = ["item_cnt_month"]
group.reset_index(inplace=True)
# Create a column in "both_ds" with the quantities sold by month, shop, item
both_ds = pd.merge(both_ds, group, on=key_cols, how='left')
display(both_ds[10000:10005])
both_ds["item_cnt_month"] = both_ds["item_cnt_month"].fillna(0).clip(0,20).astype(np.float16)
display(both_ds[10000:10005])
pct()                             

24/08/19 21:38:39


,date_block_num,shop_id,item_id,item_cnt_month
10000,0,1,6028,NaN
10001,0,1,6030,NaN
10002,0,1,6031,NaN
10003,0,1,6033,NaN
10004,0,1,6036,NaN


,date_block_num,shop_id,item_id,item_cnt_month
10000,0,1,6028,0.0
10001,0,1,6030,0.0
10002,0,1,6031,0.0
10003,0,1,6033,0.0
10004,0,1,6036,0.0


24/08/19 21:38:46


In [12]:
# 3.1.3 Add "test_ds" into "both_ds"
pct()
display(both_ds.shape)
# -- create the month column with value 34
test_ds["date_block_num"] = 34
# -- Concatenate test dataset into "both_ds" and fill NA with 0
both_ds = pd.concat([both_ds, test_ds], ignore_index=True, sort=False, keys=key_cols)
#both_ds = pd.concat([both_ds, test_ds], ignore_index=True, keys=key_cols)
both_ds.fillna(0, inplace=True) 
# -- Drop the "ID"column created in the concatenation - we don't need it
both_ds.drop("ID", axis=1, inplace=True)
#
display(both_ds.shape)
display(both_ds[10000:10000+5])
display(both_ds[10913804:10913804+5])
print(both_ds.columns)
pct()  

24/08/19 21:38:46


(10913804, 4)

(11128004, 4)

,date_block_num,shop_id,item_id,item_cnt_month
10000,0,1,6028,0.0
10001,0,1,6030,0.0
10002,0,1,6031,0.0
10003,0,1,6033,0.0
10004,0,1,6036,0.0


,date_block_num,shop_id,item_id,item_cnt_month
10913804,34,5,5037,0.0
10913805,34,5,5320,0.0
10913806,34,5,5233,0.0
10913807,34,5,5232,0.0
10913808,34,5,5268,0.0


Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month'], dtype='object')
24/08/19 21:38:47


#### feature engineering: Add fetaure - item category identification

In [13]:
# 3.2 Add fetaure: item category identification
# ---------------------------------------------
# 3.2.1 Prepare category to be inserted into the train and test datasets: create columns for category type
# --------------------------------------------------------------------------------------------------------
from sklearn.preprocessing import LabelEncoder
pct()
cats = item_categories_ds.copy()
cats['split'] = cats['item_category_name'].str.split('-')
cats['type'] = cats['split'].map(lambda x: x[0].strip())
cats['cat_type_code'] = LabelEncoder().fit_transform(cats['type'])
# if subtype is nan then type
#cats['subtype'] = cats['split'].map(lambda x: x[1].strip() if len(x) > 1 else x[0].strip())
#cats['subtype_code'] = LabelEncoder().fit_transform(cats['subtype'])
cats = cats[['item_category_id','cat_type_code']]
#cats = cats[['item_category_id','type_code', 'subtype_code']]
print(type(cats), cats.shape)
display(cats[45:50])
pct()

24/08/19 21:38:47
<class 'pandas.core.frame.DataFrame'> (84, 2)


,item_category_id,cat_type_code
45,45,12
46,46,12
47,47,12
48,48,12
49,49,12


24/08/19 21:38:47


In [14]:
# 3.2.2 Add column with "item_category_id" and "cat_type_code" corresponding to "item_id"
# ---------------------------------------------------------------------------------------
pct()
both_ds = pd.merge(both_ds, items_ds[["item_id", "item_category_id"]] , on=['item_id'], how='left')
both_ds = pd.merge(both_ds, cats, on=["item_category_id"], how="left")
both_ds["item_category_id"] = both_ds["item_category_id"].astype(np.int16)
both_ds["cat_type_code"] = both_ds["cat_type_code"].astype(np.int16)
print(both_ds.dtypes)
display(both_ds[0:5])
pct()

24/08/19 21:38:47
date_block_num        int64
shop_id               int64
item_id               int64
item_cnt_month      float16
item_category_id      int16
cat_type_code         int16
dtype: object


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code
0,0,0,19,0.0,40,11
1,0,0,27,0.0,19,5
2,0,0,28,0.0,30,8
3,0,0,29,0.0,23,5
4,0,0,32,6.0,40,11


24/08/19 21:38:52


In [15]:
# 3.3 Adding the city from the shop_id 
# ---------------------------------------------------------
pct()
if include_city_code:
    shops_ds.loc[shops_ds.shop_name == 'Сергиев Посад ТЦ "7Я"', 'shop_name'] = 'СергиевПосад ТЦ "7Я"'
    shops_ds['city'] = shops_ds['shop_name'].str.split(' ').map(lambda x: x[0])
    shops_ds.loc[shops_ds.city == '!Якутск', 'city'] = 'Якутск'
    shops_ds['city_code'] = LabelEncoder().fit_transform(shops_ds['city'])
    shops_ds = shops_ds[['shop_id','city_code']]
    both_ds = pd.merge(both_ds, shops_ds[["shop_id", "city_code"]] , on=['shop_id'], how='left')
    both_ds["city_code"] = both_ds["city_code"].astype(np.int16)
    print(both_ds.dtypes)
    display(both_ds[10000:10005])
    pct()

24/08/19 21:38:52
date_block_num        int64
shop_id               int64
item_id               int64
item_cnt_month      float16
item_category_id      int16
cat_type_code         int16
city_code             int16
dtype: object


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,city_code
10000,0,1,6028,0.0,58,13,29
10001,0,1,6030,0.0,59,13,29
10002,0,1,6031,0.0,55,13,29
10003,0,1,6033,0.0,58,13,29
10004,0,1,6036,0.0,58,13,29


24/08/19 21:38:54


In [16]:
# 3.4 Add item_name as a resulting from tdidf vectorization
# ------------------------------------------------------------------------------
pct()
if include_item_name:
    from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
    items_ds["item_name"].fillna(value="missing", inplace=True)
    vec = TfidfVectorizer(max_features=item_name_vec_size, ngram_range=(1, 3))
    item_name_sp = vec.fit_transform(items_ds['item_name'])
    item_name_ds = pd.DataFrame(item_name_sp.toarray(), columns=["item_name_"+str(i) for i in range(1, item_name_vec_size+1)])
    item_name_ds["item_id"] = items_ds["item_id"]
    both_ds = pd.merge(both_ds, item_name_ds, on=["item_id"], how="left")
    print(both_ds.shape)
    display(both_ds[0:5])
    print(item_name_sp.shape, item_name_ds.shape)
    del vec, item_name_sp
pct()

24/08/19 21:38:54
(11128004, 67)


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,city_code,item_name_1,item_name_2,item_name_3,...,item_name_51,item_name_52,item_name_53,item_name_54,item_name_55,item_name_56,item_name_57,item_name_58,item_name_59,item_name_60
0,0,0,19,0.0,40,11,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,27,0.0,19,5,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,28,0.0,30,8,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,29,0.0,23,5,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,32,6.0,40,11,29,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(22170, 60) (22170, 61)
24/08/19 21:39:02


In [17]:
# 3.5 Function to add lag features
# --------------------------------
# Function to create new columns that lags from the original column
pct()
def lag_feature(df, keys, lags, col):
    """
    Create new columns that are lags from the original column.
    "df" is the dataframe 
    "keys" list of features that are the keys - the first key is the variable that represents the time for the lag reference
    "lags" is a list with the lag values
    "col" is the column name to be lagged
    Return:
    the dataframe with N additional columns - each column corresponds to one lag 
    """
    tmp = df[keys+[col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = keys + [col + '_lag_' + str(i)]
        shifted[keys[0]] += i
        df = pd.merge(df, shifted, on=keys, how='left')
    return df
print("lag_feature compiled")


24/08/19 21:39:02
lag_feature compiled


In [18]:
# 3.6 Create lags for "item_cnt_month" (the total of itens sold per month)
pct()
if include_item_cnt_lag:
    both_ds = lag_feature(both_ds, key_cols, item_cnt_month_ll, 'item_cnt_month')
    pct()

24/08/19 21:39:02
24/08/19 21:41:40


In [19]:
pct()
display(both_ds.shape)
print(both_ds.columns)

24/08/19 21:41:41


(11128004, 72)

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
       'item_name_5

In [20]:
# 3.7 Add features - average of item counts per month
# ---------------------------------------------------
# mean item_cnt_month repeated for the entire month 
pct()
if include_date_avg_item_cnt:
    group = both_ds.groupby(['date_block_num']).agg({'item_cnt_month': ['mean']})
    group.columns = ['date_avg_item_cnt']
    group.reset_index(inplace=True)
#
    both_ds = pd.merge(both_ds, group, on=['date_block_num'], how='left')
    both_ds['date_avg_item_cnt'] = both_ds['date_avg_item_cnt'].astype(np.float16)
    both_ds = lag_feature(both_ds, key_cols, date_avg_item_cnt_ll, 'date_avg_item_cnt')
    both_ds.drop(['date_avg_item_cnt'], axis=1, inplace=True)
    pct()


24/08/19 21:41:41
24/08/19 21:42:39


In [21]:
# 3.8 Add features - average of item counts per item per month
pct()
if include_date_item_avg_item_cnt:
    group = both_ds.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month': ['mean']})
    group.columns = ["date_item_avg_item_cnt"]
    group.reset_index(inplace=True)
#
    both_ds = pd.merge(both_ds, group, on=['date_block_num','item_id'], how='left')
    print("both_ds.columns", both_ds.columns )
    both_ds['date_item_avg_item_cnt'] = both_ds['date_item_avg_item_cnt'].astype(np.float16)
    both_ds = lag_feature(both_ds,  key_cols, date_item_avg_item_cnt_ll, 'date_item_avg_item_cnt')
    both_ds.drop(['date_item_avg_item_cnt'], axis=1, inplace=True)
    print(both_ds.columns)
    pct()

24/08/19 21:42:40
both_ds.columns Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', '

In [22]:
# 3.9 Add features - average of item counts per shop per month
# -----------------------------------------------------------------------------
# Add item avg per date for all shoppings
pct()
if include_date_item_avg_item_cnt:
    group = both_ds.groupby(['date_block_num', 'shop_id']).agg({'item_cnt_month': ['mean']})
    group.columns = [ 'date_shop_avg_item_cnt' ]
    group.reset_index(inplace=True)
#
    both_ds = pd.merge(both_ds, group, on=['date_block_num','shop_id'], how='left')
    both_ds['date_shop_avg_item_cnt'] = both_ds['date_shop_avg_item_cnt'].astype(np.float16)
    both_ds = lag_feature(both_ds, key_cols, date_shop_avg_item_cnt_ll, 'date_shop_avg_item_cnt')
    both_ds.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)
#
    group_shop = group.copy()  # save to work with category ahead
#
    print(both_ds.columns)
    pct()

24/08/19 21:45:23
Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
 

In [23]:
# 3.10 Add features - average of items per category
# ----------------------------------------------------------------------
#
# Include "date_cat_avg_item_cnt" the average item count per date and category 
pct()
if include_date_cat_avg_item_cnt:
    group = both_ds.groupby(["date_block_num", "item_category_id"]).agg({"item_cnt_month": ['mean']})
    group.columns = ["date_cat_avg_item_cnt"]
    group.reset_index(inplace=True)
#
    both_ds = pd.merge(both_ds, group, on=["date_block_num","item_category_id"], how="left")
    both_ds["date_cat_avg_item_cnt"] = both_ds["date_cat_avg_item_cnt"].astype(np.float16)
    both_ds = lag_feature(both_ds, key_cols, date_cat_avg_item_cnt_ll, "date_cat_avg_item_cnt")
    both_ds.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)
#
    group_cat = group.copy()  # save to work with category ahead
#
    display(both_ds.shape)
    print(both_ds.columns)
    pct()

24/08/19 21:48:00


(11128004, 85)

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
       'item_name_5

In [24]:
# 3.11 Add items sold per date, shop and  category
# ---------------------------------------------------------------------
# Include "date_shop_cat_avg_item_cnt" the average item count per date, shop and category 
pct()
if include_date_shop_cat_avg_item_cnt:
    group = both_ds.groupby(["date_block_num", "shop_id", "item_category_id"]).agg({"item_cnt_month": ['mean']})
    group.columns = ["date_shop_cat_avg_item_cnt"]
    group.reset_index(inplace=True)
#
    both_ds = pd.merge(both_ds, group, on=["date_block_num","shop_id", "item_category_id"], how="left")
    both_ds["date_shop_cat_avg_item_cnt"] = both_ds["date_shop_cat_avg_item_cnt"].astype(np.float16)
    both_ds = lag_feature(both_ds, key_cols, date_shop_cat_avg_item_cnt_ll, "date_shop_cat_avg_item_cnt")
    both_ds.drop(['date_shop_cat_avg_item_cnt'], axis=1, inplace=True)
    display(both_ds.shape)
    print(both_ds.columns)
    pct()

24/08/19 21:49:45


(11128004, 86)

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
       'item_name_5

#### Feature engineering: Add feature - average price (transformed)  

In [25]:
# 3.12 Add feature - average price (transformed)  
# -------------------------------------------------------------------
# Handle the price 
pct()
if transform_price_log10:
    train_ds["item_price"] = np.log10(train_ds["item_price"])    
if include_date_price_avg_item:
    group = train_ds.groupby(["date_block_num", "shop_id", "item_id"]).agg({"item_price": ['mean']})
    group.columns = ["date_shop_item_avg_price"]
    group.reset_index(inplace=True)

    both_ds = pd.merge(both_ds, group, on=["date_block_num","shop_id", "item_id"], how="left")
    both_ds["date_shop_item_avg_price"] = both_ds["date_shop_item_avg_price"].astype(np.float16)
    both_ds = lag_feature(both_ds, key_cols, date_shop_item_avg_price_ll, "date_shop_item_avg_price")
    both_ds.drop(["date_shop_item_avg_price"], axis=1, inplace=True)
    display(both_ds.shape)
    print(both_ds.columns)
    pct()

24/08/19 21:50:41


(11128004, 88)

Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
       'item_name_5

In [26]:
# 3.13 eature engineering: Add feature - month of an year cycle 
# -------------------------------------------------------------
# Handle "month" attribute
pct()
# Create the "month" feature - this might be important to characterize an year as a period 
both_ds["month"] = both_ds["date_block_num"] % 12
#
# If option is one-hot encode
if month_onehotencoder:
    # Create the encoder (ohe) and transform month in ohe
    ohe = OneHotEncoder(sparse=False)
    month_ohe_array = ohe.fit_transform(both_ds["month"].values.reshape(-1,1))
    # Create a dataframe to append to "both_ds"
    month_ohe_ds = pd.DataFrame(data=month_ohe_array, columns=["month_"+str(i) for i in range(12)])
    both_ds[month_ohe_ds.columns] = month_ohe_ds
    # Drop"month" feature
    both_ds.drop(["month"], axis=1, inplace=True)
#    
    print(both_ds.shape)
    print(both_ds.columns)
pct()  

24/08/19 21:52:32


C:\Users\vanin\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(11128004, 100)
Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3', 'item_name_4', 'item_name_5',
       'item_name_6', 'item_name_7', 'item_name_8', 'item_name_9',
       'item_name_10', 'item_name_11', 'item_name_12', 'item_name_13',
       'item_name_14', 'item_name_15', 'item_name_16', 'item_name_17',
       'item_name_18', 'item_name_19', 'item_name_20', 'item_name_21',
       'item_name_22', 'item_name_23', 'item_name_24', 'item_name_25',
       'item_name_26', 'item_name_27', 'item_name_28', 'item_name_29',
       'item_name_30', 'item_name_31', 'item_name_32', 'item_name_33',
       'item_name_34', 'item_name_35', 'item_name_36', 'item_name_37',
       'item_name_38', 'item_name_39', 'item_name_40', 'item_name_41',
       'item_name_42', 'item_name_43', 'item_name_44', 'item_name_45',
       'item_name_46', 'item_name_47', 'item_name_48', 'item_name_49',
   

In [27]:
# 3.14 Add 2 features: time in months since first sale for each pair shop/item and for each item

pct()
if include_item_shop_first_sale:
    both_ds['item_shop_first_sale'] = both_ds['date_block_num'] - \
                                      both_ds.groupby(['item_id','shop_id'])['date_block_num'].transform('min')
    both_ds['item_first_sale'] = both_ds['date_block_num'] - \
                                 both_ds.groupby('item_id')['date_block_num'].transform('min')
    pct()

24/08/19 21:53:53
24/08/19 21:53:57


In [28]:
# 3.15 Function to add lag features for first time attributes
# -----------------------------------------------------------
# Treatment for new items in very first time - add features with mean of category and features
# with shop-id (See later about including category type)
#
# Function to create new columns that lags from the original column for "first_time" columns
pct()
def lag_feature_ft(df, key, lags, col):
    """
    Create new columns that are lags from the original column.
    "df" is the dataframe 
    "lags" is a list with the lag values
    "col" is the column name
    Return:
    the dataframe with N additional columns - each column corresponds to one lag 
    """
    tmp = df[['date_block_num', key, col]]
    for i in lags:
        shifted = tmp.copy()
        shifted.columns = ['date_block_num', key, col+'_lag_ft_'+str(i)]
        shifted['date_block_num'] += i
        df = pd.merge(df, shifted, on=['date_block_num', key], how='left')
    return df
print("lag_feature_ft compiled")

24/08/19 21:53:57
lag_feature_ft compiled


In [29]:
# 3.16 Create features that represent the item_category_id average at first time
# ------------------------------------------------------------------------------
# This is to improve the training for the items that appear as first time in the prediction 
# Select the items in the very first time
pct()
if include_date_cat_avg_ft:
    both_ft = both_ds[both_ds["date_block_num"] > max_lag-1]
    both_ft2 = both_ft[both_ft['item_first_sale'] == 0]
    print (both_ds.shape, both_ft.shape, both_ft2.shape)
    display(both_ft2[100000:100005])
# Initialize the new columns with lags for the average item count per category_id
    group_cat2 = lag_feature_ft (group_cat, "item_category_id",
                                 first_time_cat_ll, "date_cat_avg_item_cnt")
# Merge the new columns to "both_ds" -- observe if it will be necessary to merge only for firts-time       
    both_ds2 = pd.merge(both_ds, group_cat2, on=["date_block_num", "item_category_id"], how="left")
    both_ds2.drop(['date_cat_avg_item_cnt'], axis=1, inplace=True)
    print(both_ds2.shape)
    print(both_ds2.columns)
    display(both_ds2[100000:100005])
else:
    both_ds2=both_ds
pct()

24/08/19 21:53:57
(11128004, 102) (6639294, 102) (340207, 102)


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,city_code,item_name_1,item_name_2,item_name_3,...,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,item_shop_first_sale,item_first_sale
6909117,19,18,7212,1.0,19,5,11,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
6909138,19,18,7275,0.0,55,13,11,0.0,0.0,0.721804,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
6909168,19,18,7408,0.0,56,13,11,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
6909207,19,18,7553,0.0,75,15,11,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0
6909238,19,18,7791,6.0,19,5,11,0.0,0.0,0.000000,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,0


(11128004, 107)
Index(['date_block_num', 'shop_id', 'item_id', 'item_cnt_month',
       'item_category_id', 'cat_type_code', 'city_code', 'item_name_1',
       'item_name_2', 'item_name_3',
       ...
       'month_9', 'month_10', 'month_11', 'item_shop_first_sale',
       'item_first_sale', 'date_cat_avg_item_cnt_lag_ft_1',
       'date_cat_avg_item_cnt_lag_ft_2', 'date_cat_avg_item_cnt_lag_ft_3',
       'date_cat_avg_item_cnt_lag_ft_6', 'date_cat_avg_item_cnt_lag_ft_12'],
      dtype='object', length=107)


,date_block_num,shop_id,item_id,item_cnt_month,item_category_id,cat_type_code,city_code,item_name_1,item_name_2,item_name_3,...,month_9,month_10,month_11,item_shop_first_sale,item_first_sale,date_cat_avg_item_cnt_lag_ft_1,date_cat_avg_item_cnt_lag_ft_2,date_cat_avg_item_cnt_lag_ft_3,date_cat_avg_item_cnt_lag_ft_6,date_cat_avg_item_cnt_lag_ft_12
100000,0,15,8158,0.0,38,11,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
100001,0,15,8159,0.0,38,11,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
100002,0,15,8160,0.0,40,11,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
100003,0,15,8161,0.0,40,11,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN
100004,0,15,8167,0.0,55,13,9,0.0,0.0,0.0,...,0.0,0.0,0.0,0,0,NaN,NaN,NaN,NaN,NaN


24/08/19 21:57:04


In [30]:
# 3.17 Create features that represent the shop_id average at firts time 
# ---------------------------------------------------------------------
# This is to improve the training for the items that appear as first time in the prediction 
pct()
# Select the items in the very first time
if include_date_shop_avg_ft:
    both_ft = both_ds2[both_ds["date_block_num"] > max_lag-1]
    both_ft2 = both_ft[both_ft['item_shop_first_sale'] == 0]
    print (both_ds2.shape, both_ft.shape, both_ft2.shape)
    display(both_ft2[100000:100005])
# Initialize the new columns with lags for the average item count per category_id
    group_shop2 = lag_feature_ft (group_shop, "shop_id", first_time_shop_ll,
                                  "date_shop_avg_item_cnt")
# Merge the new columns to "both_ds" -- observe if it will be necessary to merge only for firts-time       
    both_ds3 = pd.merge(both_ds2, group_shop2, on=["date_block_num", "shop_id"], how="left")
    both_ds3.drop(['date_shop_avg_item_cnt'], axis=1, inplace=True)
    print(both_ds3.shape)
    print(both_ds3.columns)
    display(both_ds3[100000:100005])
else:
    both_ds3=both_ds2
pct()

24/08/19 21:57:04
24/08/19 21:57:04


In [31]:
# 3.18 Fill NA with 0.0 
# ---------------------
pct()
both_ds3.fillna(0.0, inplace=True)
pct()

24/08/19 21:57:04
24/08/19 21:57:10


### 4. Build the train, validation and test datasets for the first level models

In [32]:
# 4.1 Functions to split the entire dataset in train, validation and test datasets
# --------------------------------------------------------------------------------
# Set train, validation and test
pct()
def split_train_val(both_ds3, train_val_month, drop_cols=[]):
#
    #-- the training data are composed by the records before the month given by "train_val_month"
    ref = train_val_month
    X_train = both_ds3[both_ds3.date_block_num < ref].drop(['item_cnt_month'] + drop_cols, axis=1)
    y_train = both_ds3[both_ds3.date_block_num < ref]['item_cnt_month']
    #-- the validation data are composed by the records which month is "train_val_month"
    X_valid = both_ds3[both_ds3.date_block_num == ref].drop(['item_cnt_month'] + drop_cols, axis=1)
    y_valid = both_ds3[both_ds3.date_block_num == ref]['item_cnt_month']
    print("X_train.shape, y_train.shape, X_valid.shape, y_valid.shape =", X_train.shape, y_train.shape, X_valid.shape,
        y_valid.shape)
    return X_train, y_train, X_valid, y_valid
#
print("split_train_val compiled")
      
def get_X_test(both_ds3, test_month=34, drop_cols=[]):
    #-- the test data are composed by the records which month is 34 (built from the original test input file)
    X_test = both_ds3[both_ds3.date_block_num == test_month].drop(['item_cnt_month']+drop_cols, axis=1)
    print("X_test.shape =", X_test.shape)
    return X_test
print("get_X_test compiled")      

24/08/19 21:57:10
split_train_val compiled
get_X_test compiled


In [33]:
# 4.2 Set train, validation and test for first level models
# ---------------------------------------------------------
pct()
both_ds3 = both_ds3[both_ds3.date_block_num > max_lag-1]
if model_option in first_level_model_options:
    #-- Delete the first records because they do not have the lag features   
    if only_train_test:
        #-- the training data are composed by the records before the month 34
        X_train = both_ds3[both_ds3.date_block_num < 34].drop(['item_cnt_month'], axis=1)
        y_train = both_ds3[both_ds3.date_block_num < 34]['item_cnt_month']
        print("X_train.shape, y_train.shape =", X_train.shape, y_train.shape)
    else:
        #-- the training data are composed by the records before the month given by "validation_month"
        print("split for validation_month =", validation_month)
        X_train = both_ds3[both_ds3.date_block_num < validation_month].drop(['item_cnt_month'], axis=1)
        y_train = both_ds3[both_ds3.date_block_num < validation_month]['item_cnt_month']
        #-- the validation data are composed by the records which month given by "validation_month"
        X_valid = both_ds3[both_ds3.date_block_num == validation_month].drop(['item_cnt_month'], axis=1)
        y_valid = both_ds3[both_ds3.date_block_num == validation_month]['item_cnt_month']
        print("X_train.shape, y_train.shape, X_valid.shape, y_valid.shape =", X_train.shape, y_train.shape, X_valid.shape,
          y_valid.shape)
    #-- the test data are composed by the records which month is 34 (built from the original test input file)
    X_test = both_ds3[both_ds3.date_block_num == 34].drop(['item_cnt_month'], axis=1)
    print("X_test.shape =", X_test.shape)
#
pct()


24/08/19 21:57:10
24/08/19 21:57:17


In [34]:
# 4.2 Drop columns that will not be part of the fit and predict process
# ---------------------------------------------------------------------
pct()  
#
if model_option in first_level_model_options:
    def drop_feature(fea):
        X_train.drop([fea], axis=1, inplace=True)
        X_valid.drop([fea], axis=1, inplace=True)
        X_test.drop([fea], axis=1, inplace=True)   
    # Drop "date_block_num" if setting is True
    if drop_date_block_num:
        drop_feature("data_block_num")
        print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape)
    # Drop "shop_id" if setting is True
    if drop_shop_id:
        drop_feature("shop_id")
        print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape)
    # Drop item_id" if setting is True
    if drop_item_id:
        drop_feature("item_id")
    print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape)
    pct()

24/08/19 21:57:17


In [35]:
# 4.3 Garbage Collection - free memory 
# ------------------------------------
pct()
exec_gc = True
if exec_gc:
    print(psutil.virtual_memory())
    del train_ds 
    del test_ds
    del both_ds
    del both_ds2
    if model_option in first_level_model_options:   # both_ds3 is still used in tranning ensemble
        del both_ds3
    tm = TimeServ("gc.collect")
    gc.collect()
    tm.exit()
    print(psutil.virtual_memory())
    pct()

24/08/19 21:57:17
svmem(total=12748468224, available=446218240, percent=96.5, used=12302249984, free=446218240)
gc.collect execution time 0.863 sec.
svmem(total=12748468224, available=5861277696, percent=54.0, used=6887190528, free=5861277696)
24/08/19 21:57:21


### 5. Instantiate, fit and calculate the predictions for the first level models

In [36]:
# 5.1 Support functions
# ---------------------
pct()
def RMSE (pred, y):
    # -- compute the RMSE error given the preditiction and refernce values 
    # -- IMPORTANT: the parameters should be in the original scale (not transformed, for instane by log10))
    assert pred.shape[0] == y.shape[0]
    return np.sqrt(np.sum((pred - y)**2) / pred.shape[0])
print ("RMSE compiled")

def RMSE2 (pred, y):
    # -- compute the RMSE error given the preditiction and refernce values individually, returning a vector of error values 
    assert pred.shape[0] == y.shape[0]
    return (pred - y)**2 
print ("RMSE2 compiled")

def predict_and_print (model, Xtrain_ds, ytrain_ds, Xtest_ds, ytest_ds):
    # -- predict train and test and print the error results for 
    # ---- train prediction and test prediction 
    # -- return the train and test prediction to be used later
    ypred_train_ds = model.predict(Xtrain_ds)
    ypred_test_ds = model.predict(Xtest_ds)
    train_error = RMSE (ytrain_ds, ypred_train_ds)
    test_error = RMSE (ytest_ds, ypred_test_ds)
    print ("train error = {:.4f} - test error = {:.4f}".format(train_error, test_error))
    return ypred_train_ds, ypred_test_ds    
print ("predict_and_print compiled")

24/08/19 21:57:21
RMSE compiled
RMSE2 compiled
predict_and_print compiled


In [37]:
# 5.2 LightGBM training funtion
# -----------------------------
pct()
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
import lightgbm as lgb
#
def lgb_train(override_params, return_type="scores"):
    """
    train and validate a lightGBM model (proprietary interface) given
    "X_train" and "y_train" - for training
    "X_valid" and "y_valid" - for validation
    p - a dictionay with the parameters for the algorithm
    returns
    the pair (error training, error validation)
    """
    
    d_train = lgb.Dataset(X_train, label=y_train)
    d_valid = lgb.Dataset(X_valid, label=y_valid)
    watchlist = [d_train, d_valid]

    params = {
        "application": "regression",
        "boosting": "gbdt",   
        "learning_rate": 0.05,
        "num_leaves": 31,
        "max_depth": 5,
        "min_data_leaf": 20,
        "min_sum_hessian_in_leaf": 0.001,
        "feature_fraction": 0.6,
        "bagging_fraction": 1.0,
        "bagging_freq": 0,
        "lambda_l1": 0.1,
        "lambda_l2": 0.01,
        "min_gain_to_split": 0,
        "max_bin": 2047,
        "verbosity": -1,
        "metric": 'RMSE',        
        "nthread": -1
    }
    print("override parameters: ", override_params)
    if override_params:
        for k in override_params:
            if k in params:
                params[k] = override_params[k]
            else:
                print ("{} should be in the registered parameters - execution cancelled".format(k))
                return
    print("parameters =", params)
    ts = TimeServ("lightGBM_train")
    model = lgb.train(params, train_set=d_train, num_boost_round=7000,
                      valid_sets=watchlist, early_stopping_rounds=1000, verbose_eval=1000) 
    print("model.fit finalized")
    train_time = ts.exit()
    ts = TimeServ("lightGBM_scores")
    y1 = model.predict(X_train)
    train_error = RMSE(y1, y_train)
    print ("training error = {:.4f}".format(train_error))
    y = model.predict(X_valid)
    valid_error = RMSE(y, y_valid)
    print ("validation error = {:.4f}".format(valid_error))
    ts.exit()
    if return_type == "scores":
        additional_measures = {"error training": train_error, "time": train_time/60}
        return (valid_error, additional_measures)
    return model
pct()
print ("lgb_train compiled")

24/08/19 21:57:21
24/08/19 21:57:22
lgb_train compiled


In [38]:
# 5.3 LightGBM training
# ---------------------
pct()
if model_option == "lgb":
    params = {"max_depth": 6,  "min_data_leaf": 100, "bagging_fraction": 0.5, "lambda_l2": 0.1}
    lgb_model = lgb_train(params, return_type="model")
    pct()


24/08/19 21:57:22


In [39]:
# 5.4 Save the lightGBM model
# ---------------------------
import pickle
pct()
if model_option == "lgb":
    fn = "./models_to_ensemble/" + model_name + ".dat"
    pickle.dump(lgb_model, open(fn, "wb"))
    print(fn, "lgb_model pickle file saved")
    pct()

24/08/19 21:57:22


In [40]:
# Plot feature importance
pct()
if model_option == "lgb":
    ax = lgb.plot_importance(lgb_model, figsize=(10,14), max_num_features=20)
    plt.show()
    model = lgb_model   # set model to be used to generate data to submit

24/08/19 21:57:22


In [41]:
# 5.5 xgbosst training function
# -----------------------------
pct()
from xgboost import XGBRegressor
from xgboost import plot_importance
report_valid_errors=True
#
def xgb_train(override_params, return_type="scores", only_train_test=False, print_errors=True):
    """
    train and validate a XGBOOST model (proprietary interface) given
    "X_train" and "y_train" - for training
    "X_valid" and "y_valid" - for validation
    override_params - a dictionay with the parameters for the algorithm - if None, it uses the default
    return_type - "scores" return the scores to be used in the tuning process; otherwise return the model fitted
    print_errors - if True compute the prediction and errors for training and validation.
    returns
    the scores if return type == "scores" or the model fitted.
    """    
    params = { 
        "max_depth": 40,                # 8       
        "learning_rate": 0.1,           # 0.1
        "n_estimators": 100,            # 1000
        "min_child_weight": 100,        # 300
        "colsample_by_tree": 0.35,      # 0.8
        "colsample_by_level": 1.0 ,     # N/A
        "colsample_by_node": 1.0,       # N/A
        "subsample": 1.0,               # 0.8
        "reg_alpha": 0.1,
        "reg_lambda": 0.8
    }
    print("override parameters: ", override_params)
    if override_params:
        for k in override_params:
            if k in params:
                params[k] = override_params[k]
            else:
                print ("{} should be in the registered parameters - execution cancelled".format(k))
                return
    print("parameters =", params)
    ts = TimeServ("xgboost_train")
    
    n_estimators = n_estimators_only_train_test if only_train_test else params["n_estimators"]
    
    model = XGBRegressor(max_depth=params["max_depth"],
                         n_estimators=n_estimators,
                         min_child_weight=params["min_child_weight"], 
                         colsample_bytree=params["colsample_by_tree"], 
                         colsample_bylevel=params["colsample_by_level"], 
                         colsample_bynode=params["colsample_by_node"], 
                         subsample=params["subsample"],
                         reg_alpha=params["reg_alpha"],
                         reg_lambda=params["reg_lambda"],
                         silent = True,
                         seed=6)
    print("model to be fitted: ", model)
    if only_train_test:
        model.fit(X_train, y_train, eval_metric="rmse", eval_set=[(X_train, y_train)], verbose=True, early_stopping_rounds=10)
    else:            
        model.fit(X_train, y_train, eval_metric="rmse", eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=True,
                  early_stopping_rounds = 10)
    print("model.fit finalized")
    train_time = ts.exit()
    if print_errors:
        ts = TimeServ("xgboost_scores")
        y1 = model.predict(X_train)
        train_error = RMSE(y1, y_train)
        print ("training error = {:.4f}".format(train_error))
        y = model.predict(X_valid)
        valid_error = RMSE(y, y_valid)
        print ("validation error = {:.4f}".format(valid_error))
        ts.exit()
    if return_type == "scores":
        additional_measures = {"train_error": train_error, "time": train_time/60}
        return (valid_error, additional_measures)
    if report_valid_errors and print_errors:
        report_ds = X_valid.copy()
        report_ds["y"] = y_valid
        report_ds["estimated y"] = y  
        fn = "./output/report_valid_errors_" + sct2() + ".csv"
        report_ds.to_csv(fn, index=False) 
        print(fn, "saved")
    return model
pct()
print ("xgb_train compiled")

24/08/19 21:57:22
24/08/19 21:57:22
xgb_train compiled


In [42]:
# 5.6 xgboost training
# --------------------
pct()
if model_option == "xgb":
    
    xgb_params = {                      # Best results: leaderboard = 0.88575
        "learning_rate": 0.1,           # 0.1
        "n_estimators": 100,            # 100
        "min_child_weight": 300,        # 300
        "colsample_by_tree": 0.35,      # colsample_by_tree = 0.35
        "colsample_by_level": 1.0 ,     # N/A - valor default = 1.0
        "colsample_by_node": 1.0 ,      # N/A - valor default = 1.0
        "subsample": 1.0,               # 1.0
        "reg_alpha": 0.6,               # 0.6
        "reg_lambda": 0.6               # 0.6
    }
    
    xgb_model = xgb_train(xgb_params, return_type="model", only_train_test= only_train_test, print_errors=False)
    ts = TimeServ("xgboost_scores")
#    y1 = xgb_model.predict(X_train)
#    train_error = RMSE(y1, y_train)
#    print ("training error = {:.4f}".format(train_error))
    y = xgb_model.predict(X_valid)
    valid_error = RMSE(y, y_valid)
    print ("validation error = {:.4f}".format(valid_error))
    ts.exit()
    pct()


24/08/19 21:57:22


In [43]:
# 5.7 xgboost - save the model
import pickle
pct()
if model_option == "xgb":
    fn = "./models_to_ensemble/" + model_name + ".dat"
    pickle.dump(xgb_model, open(fn, "wb"))
    print(fn, "xgb_model pickle file saved")
    pct()


24/08/19 21:57:22


In [44]:
# Plot feature importance
pct()
if model_option == "xgb":
    def plot_features(booster, figsize):    
        fig, ax = plt.subplots(1,1,figsize=figsize)
        return plot_importance(booster=booster, ax=ax, max_num_features=24)
    print("plot_features compiled")
    plot_features(xgb_model, (10,14))
    model = xgb_model   # set model to be used to generate data to submit

24/08/19 21:57:22


In [45]:
# Garbage collect
pct()
tm = TimeServ("gc.collect")
gc.collect()
tm.exit()
print(psutil.virtual_memory())
pct()

24/08/19 21:57:22
gc.collect execution time 0.032 sec.
svmem(total=12748468224, available=5809557504, percent=54.4, used=6938910720, free=5809557504)
24/08/19 21:57:22


### 6. Build the dataset for ensemble models

In [46]:
# 6. Ensemble - Build the dataset for the ensemble model
# ------------------------------------------------------
# Note: it takes 35 minutes for 6 models
pct()
if model_option in  ensemble_model_options:
    # Create an empty dataframe with nrows for all train and test datasets and columns for each 1st level models + target  
    ens_df = pd.DataFrame(index=range(both_ds3.shape[0]),columns=ens_first_level_models+["item_cnt_month"])
    print("ens_df.shape =", ens_df.shape)
    # The input data to predict the first level models (in order to generate the features for the ensemble 
    # is the entire both_ds3 without the target column
    X_train_ds3 = both_ds3.drop(['item_cnt_month'], axis=1)
    # For each fitted model from first level model
    for flm in ens_first_level_models:
        # Load the model
        fn = "./models_to_ensemble/" + flm + ".dat"
        print("fn =", fn)
        model = pickle.load(open(fn, "rb"))
        print("Model {} loaded".format(flm))
        # Predict the loaded model for all data and store the result as a feature for the ensemble model
        ens_df[flm] = model.predict(X_train_ds3).clip(0,20)
    # Copy the original target to the target for the ensemble model
    ens_df["date_block_num"] = both_ds3.reset_index()["date_block_num"]
    ens_df["item_cnt_month"] = both_ds3.reset_index()["item_cnt_month"]
    display(ens_df[500000:500010])
    pct()       

24/08/19 21:57:22
ens_df.shape = (6639294, 9)
fn = ./models_to_ensemble/M1C0.dat
Model M1C0 loaded
fn = ./models_to_ensemble/M1C1.dat
Model M1C1 loaded
fn = ./models_to_ensemble/M1C2.dat
Model M1C2 loaded
fn = ./models_to_ensemble/M1C3.dat
Model M1C3 loaded
fn = ./models_to_ensemble/M2C0.dat
Model M2C0 loaded
fn = ./models_to_ensemble/M2C1.dat
Model M2C1 loaded
fn = ./models_to_ensemble/M2C2.dat
Model M2C2 loaded
fn = ./models_to_ensemble/M2C3.dat
Model M2C3 loaded


,M1C0,M1C1,M1C2,M1C3,M2C0,M2C1,M2C2,M2C3,item_cnt_month,date_block_num
500000,0.746682,0.587352,0.735646,0.607166,0.741309,0.721383,0.743945,0.771642,1.0,13
500001,0.079340,0.043193,0.093668,0.053332,0.139678,0.090111,0.119016,0.109051,0.0,13
500002,0.417083,0.388648,0.389044,0.403816,0.260229,0.269307,0.273203,0.271136,3.0,13
500003,0.884512,0.893808,0.821595,0.861104,0.739427,0.631168,0.655729,0.704724,1.0,13
500004,0.884512,0.893808,0.821595,0.861104,0.739427,0.631168,0.655729,0.704724,0.0,13
500005,0.000000,0.014500,0.017214,0.011699,0.067942,0.067937,0.073680,0.085963,0.0,13
500006,0.164432,0.172880,0.106133,0.102244,0.349041,0.262436,0.325278,0.304766,0.0,13
500007,0.112943,0.138022,0.099409,0.146592,0.210680,0.174716,0.171930,0.188375,0.0,13
500008,0.473303,0.507268,0.495858,0.479656,0.616343,0.595641,0.618688,0.573694,0.0,13
500009,0.058298,0.102940,0.042503,0.047716,0.196455,0.156283,0.190300,0.190243,0.0,13


24/08/19 22:42:25


In [47]:
# 7. Save the model results for analysis 
pct()
if model_option in ensemble_model_options:
    fn = "./output/ensemble_data_" + sct2() + ".csv"
    ens_df.to_csv(fn, index=False) 
    print(fn, "saved")
    pct()

24/08/19 22:42:25
./output/ensemble_data_190824_224225.csv saved
24/08/19 22:43:55


### 7. Split into train, valid and test datasets and instantiate, fit and calculate the predictions for ensemble models

In [48]:
# 7.1 SGD ensemble - Split into train, valid and test datasets and instantiate, 
# fit and calculate the predictions
# -----------------------------------------------------------------------------
pct()
if model_option == "ens_sgd":
    print("The ensemble model is a linear regression of the predictions from the 1st level")
    print("-------------------------------------------------------------------------------")
    from sklearn.linear_model import SGDRegressor
    # Split the dataset into training and validation
    X_train, y_train, X_valid, y_valid = split_train_val(ens_df, 33, drop_cols=["date_block_num"])
    print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
    # Prepare X_test for final prediction
    X_test = get_X_test(ens_df, drop_cols=["date_block_num"])
    # Instantiate ensemble model
    model = SGDRegressor(eta0=0.001, early_stopping=True, n_iter_no_change= 10, random_state=6, verbose=1)
    # Fit the model
    model.fit(X_train, y_train)
    # eveluate the metrics
    ts = TimeServ("ensemble_scores")
    y = model.predict(X_train)
    train_error = RMSE(y, y_train)
    print ("train error = {:.4f}".format(train_error))
    y = model.predict(X_valid)
    valid_error = RMSE(y, y_valid)
    print ("validation error = {:.4f}".format(valid_error))
    ts.exit()  
    pct()

24/08/19 22:43:55
The ensemble model is a linear regression of the predictions from the 1st level
-------------------------------------------------------------------------------
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape = (6186922, 8) (6186922,) (238172, 8) (238172,)
(6186922, 8) (6186922,) (238172, 8) (238172,)
X_test.shape = (214200, 8)
-- Epoch 1
Norm: 1.61, NNZs: 8, Bias: -0.014545, T: 5568229, Avg. loss: 0.204420
Total training time: 1.54 seconds.
-- Epoch 2
Norm: 1.76, NNZs: 8, Bias: -0.013687, T: 11136458, Avg. loss: 0.201075
Total training time: 3.58 seconds.
-- Epoch 3
Norm: 1.80, NNZs: 8, Bias: -0.014475, T: 16704687, Avg. loss: 0.200874
Total training time: 5.55 seconds.
-- Epoch 4
Norm: 1.82, NNZs: 8, Bias: -0.010499, T: 22272916, Avg. loss: 0.200793
Total training time: 7.52 seconds.
-- Epoch 5
Norm: 1.83, NNZs: 8, Bias: -0.009289, T: 27841145, Avg. loss: 0.200752
Total training time: 9.49 seconds.
-- Epoch 6
Norm: 1.84, NNZs: 8, Bias: -0.014964, T: 334093

In [53]:
# 7.2 SGD ensemble - Split into train, valid and test datasets and instantiate, 
# fit and calculate the predictions
# -----------------------------------------------------------------------------
pct()
if model_option == "ens_mean":
    print("The ensemble model is the mean among the predictions from the 1st level")
    print("-------------------------------------------------------------------------")
    from sklearn.linear_model import SGDRegressor
    # Split the dataset into training and validation
    X_train, y_train, X_valid, y_valid = split_train_val(ens_df, 33, drop_cols=["date_block_num"])
    print(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape)
    # Prepare X_test for final prediction
    X_test = get_X_test(ens_df, drop_cols=["date_block_num"])
    # eveluate the metrics
    ts = TimeServ("ensemble_scores")
    y = X_train.mean(axis=1)
    train_error = RMSE(y, y_train)
    print ("train error = {:.4f}".format(train_error))
    y = X_valid.mean(axis=1)
    valid_error = RMSE(y, y_valid)
    print ("validation error = {:.4f}".format(valid_error))
    ts.exit()  
    pct()

25/08/19 06:39:30
The ensemble model is the mean among the predictions from the 1st level
-------------------------------------------------------------------------
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape = (6186922, 8) (6186922,) (238172, 8) (238172,)
(6186922, 8) (6186922,) (238172, 8) (238172,)
X_test.shape = (214200, 8)
train error = 0.7193
validation error = 0.8660
ensemble_scores execution time 0.552 sec.
25/08/19 06:39:31


### 8. Generate the submission file. 

In [54]:
# 8.1 Predict the test data
# -------------------------
pct()
if model_option == "ens_mean":
    y_test = np.array(X_test.mean(axis=1).clip(0, 20))
else:
    y_test = model.predict(X_test).clip(0, 20)
display(X_test[0:10])
print(type(y_test))
display(y_test[0:10])
#
report_ds = X_test.copy()
report_ds["y"] = y_test
fn = "./output/report_test_prediction_" + sct2() + ".csv"
report_ds.to_csv(fn, index=False) 
print(fn, "saved")
pct()


25/08/19 06:39:31


,M1C0,M1C1,M1C2,M1C3,M2C0,M2C1,M2C2,M2C3
6425094,0.640936,0.442560,0.551458,0.607598,0.486504,0.564399,0.540958,0.722831
6425095,1.152045,1.136337,0.721992,0.917711,0.368664,0.809579,0.388517,0.473664
6425096,1.032598,1.169719,1.202327,1.799619,1.078988,1.328196,1.225420,1.580268
6425097,0.511651,0.828554,0.830577,1.136871,0.475794,0.548550,0.579186,0.709279
6425098,3.182692,2.149438,1.155115,4.606379,4.237621,2.077641,3.558893,2.974090
6425099,0.609138,0.832837,0.532136,0.878928,0.570516,0.566906,0.637117,0.652811
6425100,0.827245,1.637587,1.076229,1.205964,0.725567,0.549913,0.688897,0.721259
6425101,0.141967,0.128827,0.227706,0.161854,0.143986,0.188790,0.166671,0.177408
6425102,0.889616,0.962624,0.943885,0.829083,0.849332,0.868760,0.928735,0.902329
6425103,0.449230,0.500500,0.764303,0.938418,0.838925,0.876548,0.910291,0.821499


<class 'numpy.ndarray'>


array([0.56965537, 0.74606378, 1.30214185, 0.70255793, 2.99273352,
       0.66004843, 0.92908262, 0.16715115, 0.89679543, 0.76246425])

./output/report_test_prediction_190825_063931.csv saved
25/08/19 06:39:34


In [55]:
# 8.2 Generate the submission file
# --------------------------------
pct()
submit_ds = sample_submission_ds
submit_ds["item_cnt_month"] = y_test
display(submit_ds[0:10])
fn = "./output/submission_" + model_option + "_" + sct2() + ".csv"
submit_ds.to_csv(fn, index=False)
print("file", fn, "for submission generated")
pct()

25/08/19 06:39:34


,ID,item_cnt_month
0,0,0.569655
1,1,0.746064
2,2,1.302142
3,3,0.702558
4,4,2.992734
5,5,0.660048
6,6,0.929083
7,7,0.167151
8,8,0.896795
9,9,0.762464


file ./output/submission_ens_mean_190825_063934.csv for submission generated
25/08/19 06:39:35
